In [64]:
import os
import math
import numpy as np
import pandas as pd
import requests
import pickle
from datetime import datetime as dt
from datetime import timedelta
from pytz import timezone
from pprint import pprint
import pandas_datareader.data as web

import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
import yfinance as yf
import fundamentalanalysis as fa
import streamlit as st

from functions import *


In [3]:
end = SPY_df.iloc[-1].name.date()
n = 1
start = end - timedelta(days=365 * n)
print(f'start: {start}, end: {end}')

start: 2021-12-28, end: 2022-12-28


In [4]:
sectors, subIndustries, tickers, SPY, rf = calculate_metrics(start, end)

In [2]:
with open('data/financial_statements.pickle', 'rb') as f:
    statements = pickle.load(f)

statements['FB']

,2021-12-31,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31,2015-12-31,2014-12-31,2013-12-31,2012-12-31
Item,,,,,,,,,,
Cash and Short Term Investments,4.799800e+10,6.195400e+10,5.485500e+10,4.111400e+10,4.171100e+10,2.944900e+10,1.843400e+10,1.119900e+10,1.144900e+10,9.626000e+09
Receivables,1.403900e+10,1.133500e+10,9.518000e+09,7.587000e+09,5.832000e+09,3.993000e+09,2.559000e+09,1.678000e+09,1.109000e+09,1.170000e+09
Other current assets,4.480000e+09,2.140000e+09,1.844000e+09,1.769000e+09,1.020000e+09,9.590000e+08,NaN,NaN,NaN,NaN
Total current assets,6.666600e+10,7.567000e+10,6.622500e+10,5.048000e+10,4.856300e+10,3.440100e+10,2.165200e+10,1.339000e+10,1.307000e+10,1.126700e+10
"Property, Plant, Equpment (Net)",5.780900e+10,4.563300e+10,3.532300e+10,2.468300e+10,1.372100e+10,8.591000e+09,5.687000e+09,3.967000e+09,2.882000e+09,2.391000e+09
...,...,...,...,...,...,...,...,...,...,...
Net Cash/Marketcap,-3.540000e-02,-6.520000e-02,-7.470000e-02,-1.082000e-01,-7.910000e-02,-8.770000e-02,-6.250000e-02,-5.430000e-02,-8.480000e-02,-1.211000e-01
Assets Liabilities Change (Total),7.000000e+08,-2.723000e+09,7.250000e+09,-1.527000e+09,1.887000e+09,7.580000e+08,7.840000e+08,-2.620000e+08,6.760000e+08,-4.990000e+08
Investments Change (Net),1.213200e+10,-1.452000e+10,-4.254000e+09,2.449000e+09,-1.325000e+10,-7.186000e+09,-6.700000e+09,1.243000e+09,-8.820000e+08,-4.874000e+09


In [70]:
df_mom = web.DataReader('F-F_Momentum_Factor', 'famafrench')[0]
# df_mom.index = df_mom.index.format()
df_mom.index  

PeriodIndex(['2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06',
             '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12',
             '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06',
             '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12',
             '2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06',
             '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12',
             '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06',
             '2021-07', '2021-08', '2021-09', '2021-10', '2021-11', '2021-12',
             '2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06',
             '2022-07', '2022-08', '2022-09', '2022-10', '2022-11'],
            dtype='period[M]', name='Date')

In [3]:
def get_ticker_data(ticker):
    f = 'data/market_data'
    ticker_df = pd.read_csv(os.path.join(f, f'{ticker}.csv'), index_col='Unnamed: 0',
                            parse_dates=True)

    return ticker_df

In [4]:
df = get_ticker_data('MMM')
# df['rtn'] = df['close'].pct_change()
# df['log_rtn'] = np.log(df.close) - np.log(df.close.shift(1)) # np.log1p(df.rtn)
# df['cum_rtn'] = df.rtn.cumprod().round(6)
# df['cum_log_rtn'] = df.log_rtn.cumsum()
#
df.head()

,open,high,low,close,adjclose,volume,ticker
1970-03-25,6.375000,6.554688,6.375000,6.476563,1.303751,436800,MMM
1970-03-26,6.476563,6.546875,6.437500,6.460938,1.300605,254400,MMM
1970-03-30,6.460938,6.500000,6.414063,6.500000,1.308469,198400,MMM
1970-03-31,6.500000,6.578125,6.468750,6.531250,1.314760,224000,MMM
1970-04-01,6.562500,6.679688,6.562500,6.632813,1.335204,241600,MMM


In [5]:
def get_SPY_data():
    '''Make dataframe of SPY market data'''

    df = pd.read_csv(r'data\SPY.csv')
    df.index = pd.to_datetime(df['Date'].apply(lambda x: x.split(' ')[0]))
    df.drop(columns='Date', inplace=True)

    return df


In [6]:
SPY_info_df = pd.read_csv('data/SPY-Info.csv')
SPY_df = get_SPY_data()
SPY_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 23863 entries, 1927-12-30 to 2022-12-28
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          23863 non-null  float64
 1   High          23863 non-null  float64
 2   Low           23863 non-null  float64
 3   Close         23863 non-null  float64
 4   Volume        23863 non-null  int64  
 5   Dividends     23863 non-null  float64
 6   Stock Splits  23863 non-null  float64
dtypes: float64(6), int64(1)
memory usage: 1.5 MB


In [116]:
df = pd.read_csv(r'data\T-Bill Rates.csv', index_col='Date', parse_dates=True)
df = df[start : end].div(100)
# df['Close'].div(100)
# df['Daily T-Bill Rate'] = (1 / (1 - df['Close'] / 100 * (90 / 360)))**(1 / 90) - 1
# df['Annualised'] = (1 + df['Daily T-Bill Rate'] * 4)**4 - 1
t = len(df) / 252
# px.line(df, y='Close')
# (df['Close'][-1] / df['Close'][0]) ** (1 / t) - 1
df.loc[end:, 'Close'].item()
# df.tail()

0.04338000297546387

In [73]:
# SPY_df['Return'] = SPY_df['Close'].pct_change()
SPY_df['Return'] = np.log1p(SPY_df['Close'].pct_change())
SPY_df = SPY_df.copy()[start : end]
t = len(SPY_df) / 252
rtn = SPY_df.Return.cumsum()[-1]
cagr = ((SPY_df['Close'][-1] / SPY_df['Open'][0])**(1 / t) - 1) * 100
std = SPY_df['Return'].std() * np.sqrt(t * 252)

print(f'''
t: {t}
Cumulative Return: {rtn}
CAGR: {cagr}
StDev: {std}
        ''')


t: 1.003968253968254
Cumulative Return: -0.23519264605820173
CAGR: -21.03482825507401
StDev: 0.24123725600147436
        


In [40]:
SPY_df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Return,Cumulative Return
Date,,,,,,,,,
2021-12-29,4788.640137,4804.060059,4778.080078,4793.060059,2963310000,0.0,0.0,NaN,NaN
2021-12-30,4794.229980,4808.930176,4775.330078,4778.729980,3124950000,0.0,0.0,-0.002990,-0.002994
2021-12-31,4775.209961,4786.830078,4765.750000,4766.180176,2677820000,0.0,0.0,-0.002626,-0.002630
2022-01-03,4778.140137,4796.640137,4758.169922,4796.560059,3831020000,0.0,0.0,0.006374,0.006354
2022-01-04,4804.509766,4818.620117,4774.270020,4793.540039,4683170000,0.0,0.0,-0.000630,-0.000630


In [17]:
SPY_df['Return'] = SPY_df.Close.pct_change()
SPY_df['Cum Return'] = SPY_df['Return'].cumprod()
SPY_df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Return,Cum Return
0,1927-12-30 00:00:00-05:00,17.660000,17.660000,17.660000,17.660000,0,0.0,0.0,NaN,NaN
1,1928-01-03 00:00:00-05:00,17.760000,17.760000,17.760000,17.760000,0,0.0,0.0,0.005663,5.662536e-03
2,1928-01-04 00:00:00-05:00,17.719999,17.719999,17.719999,17.719999,0,0.0,0.0,-0.002252,-1.275375e-05
3,1928-01-05 00:00:00-05:00,17.549999,17.549999,17.549999,17.549999,0,0.0,0.0,-0.009594,1.223555e-07
4,1928-01-06 00:00:00-05:00,17.660000,17.660000,17.660000,17.660000,0,0.0,0.0,0.006268,7.669046e-10


In [56]:
d = {'a': 1, 'b': 2}
df = pd.DataFrame.from_dict(d, orient='index')
df.index.names = df.keys().to_list()
df.iloc[1]

0    2
Name: b, dtype: int64

In [104]:
first_date = SPY_df.iloc[0].name
last_date = SPY_df.iloc[-1].name
# yr_ago = last_date - timedelta(days=365)
# start_date = st.date_input('Start Date', yr_ago, min_value=first_date)
# end_date = st.date_input('End Date', last_date, max_value=last_date)
# start_date = pd.to_datetime(start_date)
# end_date = (end_date)
# SPY_df[start_date: end_date]

In [6]:
f = 'data/market_data/'

In [7]:
def get_first_dates():
    '''Get the earliest date for which market data is available for each company'''

    first_dates = []

    for file in os.listdir(f):
        df = pd.read_csv(f + file, index_col='Unnamed: 0', parse_dates=True)
        first_date = df.iloc[0].name
        ticker = file.split('.')[0]
        first_dates.append((ticker, first_date))
        first_dates = sorted(first_dates, key=lambda x: x[1])

    return first_dates

In [8]:
def make_combined_returns_df():
    '''Make dataframe of returns for all SPY stocks, as well as SPY index'''

    combined_returns = SPY_df.copy()
    combined_returns['Return'] = combined_returns['Close'].pct_change() * 100
    combined_returns.drop(
        ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits'],
        axis=1, inplace=True
        )
    combined_returns.rename(columns={'Return': 'SPY'}, inplace=True)
    
    for file in os.listdir(f):
        ticker = file.replace('.csv', '')
        df = pd.read_csv(f + file, index_col='Unnamed: 0', parse_dates=True)
        df['Return'] = df['adjclose'].pct_change() * 100
        combined_returns = combined_returns.join(df['Return'], how='left')
        combined_returns.rename(columns={'Return': ticker}, inplace=True)

    return combined_returns

In [9]:
ticker_list = SPY_info_df['Symbol'].to_list()
sector_list = SPY_info_df['GICS Sector'].unique()
last_date = SPY_df.iloc[-1].name.date()
yr_ago = last_date - timedelta(days=365)
first_dates = get_first_dates()
combined_returns_df = make_combined_returns_df()

In [13]:
combined_returns_df.tail()

,SPY,A,AAL,AAP,AAPL,ABBV,ABC,ABK,ABMD,ABT,...,XEL,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2022-08-22,-2.140006,-2.674025,-3.246295,-4.125209,-2.302937,-1.064512,-1.999601,0.0,-3.446870,-2.371434,...,-0.961796,-0.074404,-3.661257,-5.005384,-2.562037,-1.637380,-1.651508,-3.815352,-2.811522,-2.642762
2022-08-23,-0.223786,-0.970586,0.802332,0.287186,-0.202907,-0.940567,-0.651051,0.0,0.336715,-1.340153,...,-1.115484,4.244227,-0.493188,0.793208,-0.855848,-0.026145,-1.849854,0.351395,0.339283,-1.587491
2022-08-24,0.291616,0.776537,1.230102,-9.615674,0.179395,-0.798447,0.141867,NaN,2.215611,-0.537685,...,0.079635,0.591839,0.174931,NaN,-0.020805,-0.034871,0.365960,0.741350,0.160171,0.129299
2022-08-25,1.409161,1.750577,3.073626,0.600301,1.492270,1.029656,1.477435,NaN,2.391435,0.426780,...,0.888474,0.517341,1.164138,NaN,1.477172,1.874947,2.041938,3.381981,1.830140,1.783185
2022-08-26,-3.368806,-4.514374,-4.715675,-4.088629,-3.769925,-2.138804,-2.812123,NaN,-5.950088,-3.768059,...,-1.511569,-1.231198,-3.308394,NaN,-4.695030,-3.989040,-3.957477,-6.657785,-1.518064,-3.407237


In [17]:
str(dt.now().year - 1)

'2021'

In [38]:
def make_combined_ratios_df(year=str(dt.now().year - 1), period='Annual'):
    f = f'data/financial_ratios/{period}' 
    subIndustry_list = SPY_info_df['GICS Sub-Industry'].unique()  
    sector_ratios = {sector: {} for sector in sector_list}
    subIndustry_ratios = {si: {} for si in subIndustry_list}
    subIndustry_dfs = {si: pd.DataFrame() for si in subIndustry_list}
    ticker_ratios = {}

    for sector in sector_list:
        sector_tickers = SPY_info_df[SPY_info_df['GICS Sector'] == sector] \
                         ['Symbol'].to_list()
        sector_dfs = pd.DataFrame()
        for ticker in sector_tickers:
            # Get sub-industry of ticker
            t_subIndustry = SPY_info_df[SPY_info_df['Symbol'] == ticker] \
                            ['GICS Sub-Industry'].item()
            weight = ticker_weights[ticker]
            t_sector_weight = weight / sector_weights[sector]
            t_subIndustry_weight = weight / subIndustry_weights[t_subIndustry]
            df = pd.read_csv(os.path.join(f, f'{ticker}.csv'), index_col='Unnamed: 0')
            df = df.drop(index='period').fillna(0)
            try:
                df = df.loc[:, year].astype('float')
            except:
                print(ticker)
                df.loc[:, year] = 0.0
                df = df.loc[:, year]
            sector_df = df * t_sector_weight
            sector_dfs = pd.concat([sector_dfs, sector_df], axis=1)
            subIndustry_df = df * t_subIndustry_weight
            subIndustry_dfs[t_subIndustry] = pd.concat([subIndustry_dfs[t_subIndustry], 
                                                        subIndustry_df], axis=1)
            ticker_ratios[ticker] = df.to_dict()
        for ratio in df.index:
            sector_ratios[sector][ratio] = sum(sector_dfs.loc[ratio, :])
            for si in subIndustry_list:
                if not subIndustry_dfs[si].empty:
                    subIndustry_ratios[si][ratio] = sum(subIndustry_dfs[si].loc[ratio, :])

    sector_ratios = pd.DataFrame(sector_ratios)
    subIndustry_ratios = pd.DataFrame(subIndustry_ratios)
    ticker_ratios = pd.DataFrame(ticker_ratios)

    f = 'data/financial_ratios'
    sector_ratios.to_csv(os.path.join(f, f'{period}-{year} Sector Ratios.csv'))
    subIndustry_ratios.to_csv(os.path.join(f, f'{period}-{year} Sub-Industry Ratios.csv'))
    ticker_ratios.to_csv(os.path.join(f, f'{period}-{year} Ticker Ratios.csv'))
    
    return sector_ratios, subIndustry_ratios, ticker_ratios

sector_ratios, subIndustry_ratios, ticker_ratios = make_combined_ratios_df(year='2021')

In [18]:
subIndustry_ratios

,Industrial Conglomerates,Building Products,Health Care Equipment,Pharmaceuticals,IT Consulting & Other Services,Interactive Home Entertainment,Agricultural Products,Application Software,Data Processing & Outsourced Services,Automotive Retail,...,Environmental & Facilities Services,Human Resource & Employment Services,Apparel Retail,Food Distributors,Wireless Telecommunication Services,Electronic Manufacturing Services,Trading Companies & Distributors,Drug Retail,Broadcasting,Household Appliances
currentRatio,1.321404,1.925425,3.538283,1.334399,1.286023,3.797147,11.992693,1.508145,1.339062,1.321693,...,0.736494,1.669678,1.525035,1.466366,0.889017,1.559923,0.826354,0.717058,2.100029,1.139248
quickRatio,0.891004,1.065763,2.869544,0.933544,1.054240,3.440054,0.984342,1.322078,0.894949,0.334110,...,0.611721,1.180337,1.082870,0.928625,0.662709,0.887242,0.699577,0.310873,1.836080,0.721974
cashRatio,0.427059,0.372788,1.997246,0.344382,0.488138,2.919660,0.984342,0.703375,0.532233,0.176013,...,0.037862,0.455592,1.041875,0.410818,0.282182,0.258376,0.055321,0.054094,1.128939,0.357697
daysOfSalesOutstanding,71.258215,47.873654,58.845722,71.905310,64.856146,41.599238,0.000000,65.991330,119.742391,27.086972,...,46.676700,55.624132,4.968636,26.968935,40.737787,71.615627,62.999691,15.598903,73.233533,51.466909
daysOfInventoryOutstanding,84.837762,72.488201,129.705601,135.968115,6.869159,0.712208,0.000000,11.681884,0.000000,294.494031,...,0.679828,0.000000,61.997164,32.158316,25.596367,91.322738,10.209790,28.513768,0.000000,56.423817
operatingCycle,156.095978,110.743334,188.551323,207.873425,71.725305,42.311445,0.000000,77.673214,100.708032,321.581003,...,14.531225,55.624132,66.965799,59.127250,66.334153,162.938365,73.209481,44.112671,73.233533,107.890726
daysOfPayablesOutstanding,84.038528,59.859874,63.400610,92.455181,30.479066,94.137399,0.000000,126.211129,138.331782,236.249326,...,15.432036,17.816236,75.305455,42.510695,113.722852,69.501295,50.799932,38.917677,32.549784,112.411527
cashConversionCycle,72.057450,62.279211,125.150713,115.418244,41.246239,-51.825954,0.000000,-48.537915,-37.623750,85.331677,...,-0.900811,37.807896,-8.339655,16.616555,-47.388699,93.437070,22.409550,5.194994,40.683749,-4.520801
grossProfitMargin,0.389354,0.271616,0.611220,0.674388,0.416349,0.803132,0.070230,0.815112,0.444932,0.443447,...,0.156698,0.417249,0.230161,0.182400,0.543111,0.327481,0.396562,0.211812,0.621032,0.200546
operatingProfitMargin,0.172518,0.113049,0.199432,0.259817,0.130241,0.282689,0.035109,0.235181,0.426802,0.074202,...,0.171945,0.114913,0.017235,0.028018,0.086023,0.180795,0.234356,0.026270,0.165040,0.105890


In [20]:
ticker_ratios

,MMM,AOS,ALK,ALLE,AAL,AME,BA,CHRW,CARR,CAT,...,KMI,MRO,MPC,OXY,OKE,PSX,PXD,SLB,VLO,WMB
currentRatio,1.885003,1.825567,0.933147,2.195590,0.669624,2.344488,1.393698,1.598983,1.668102,1.534549,...,0.631257,1.328937,1.805976,1.072480,1.390636,1.394831,1.361490,1.231436,1.706776,0.616214
quickRatio,1.241570,1.438114,0.891218,1.587728,0.495262,1.682562,0.408352,1.571179,1.282192,1.016137,...,0.488175,1.596043,0.394241,0.501399,1.007248,0.949359,1.187828,0.786674,1.014974,2.717119
cashRatio,0.583040,0.646621,0.319124,0.921189,0.014787,1.127247,0.088818,0.132517,0.609589,0.363651,...,0.233346,0.611707,0.026496,0.244193,0.390091,0.264131,0.756558,0.080450,0.356889,0.061233
daysOfSalesOutstanding,54.777218,73.748834,49.130679,46.646568,28.253446,48.034336,63.080144,59.607486,71.866693,146.706429,...,40.337179,88.038424,30.129409,43.347465,35.456212,37.120959,36.323320,81.147197,33.963703,47.285918
daysOfInventoryOutstanding,93.178862,61.292877,5.227387,67.050483,23.627722,68.111595,467.176903,0.000000,48.156232,140.262546,...,26.973880,8.001154,42.247424,46.292683,23.802986,24.042249,13.195610,58.295714,17.689690,32.129450
operatingCycle,147.956080,135.041711,54.358066,113.697051,51.881169,116.145931,530.257046,59.607486,120.022926,286.968975,...,67.311059,96.039578,72.376833,89.640148,59.259198,61.163208,49.518930,139.442911,51.653392,79.415368
daysOfPayablesOutstanding,56.294188,121.564546,9.904523,52.200376,17.508523,43.896009,73.911314,29.007805,57.231716,75.384045,...,64.876832,88.117969,41.212233,72.853659,46.151185,34.269314,60.676243,51.047857,17.818598,113.870550
cashConversionCycle,91.661892,13.477165,44.453543,61.496675,34.372646,72.249922,456.345733,30.599681,62.791209,211.584930,...,2.434228,7.921609,31.164600,16.786489,13.108013,26.893894,-11.157313,88.395054,33.834795,-34.455182
grossProfitMargin,0.484060,0.382758,-0.116096,0.433398,-0.438138,0.339979,-0.097751,0.072153,0.292679,0.289283,...,0.597521,-0.119470,0.009616,0.159695,0.334038,0.078389,0.117882,0.110207,-0.897657,0.799845
operatingProfitMargin,0.211969,0.154699,-0.258553,0.148351,-0.638980,0.226405,-0.223151,0.041542,0.107126,0.109059,...,0.298462,-0.381014,-0.175511,-0.070638,0.159368,-0.024076,0.062073,0.070167,-0.024051,0.285270


In [74]:
x = SPY_info_df[SPY_info_df['GICS Sub-Industry'] == 'Industrial Machinery'].all(axis=1)
x.index
# df = SPY_info_df.loc[x.index, ['Symbol', 'Security']].set_index('Symbol').to_dict()
# # df = df.set_index('Symbol')
# df

Int64Index([136, 156, 206, 245, 247, 250, 340, 358, 362, 366, 420, 424, 494,
            497],
           dtype='int64')

In [19]:
missing = list(filter(lambda x: x[1] > dt(2021,8,22), first_dates))
missing = [x[0] for x in missing]
missing

['TMC', 'CEG', 'DO']

In [ ]:
pd.DataFrame(sector_ratios)

,Industrials,Health Care,Information Technology,Communication Services,Consumer Staples,Consumer Discretionary,Utilities,Financials,Materials,Real Estate,Energy
currentRatio,1.644125,1.981862,1.793047,2.298785,7.149155,1.500129,0.758147,1.037935e+02,1.572737,6.525279,1.462243
quickRatio,1.124483,1.483122,1.472431,2.124005,1.882192,0.943890,0.373296,1.155012e+04,1.007785,7.679389,-93.852236
cashRatio,0.437787,0.789175,0.440924,0.525175,1.637310,0.573609,0.097288,1.151206e+04,0.464249,1.283062,-94.532021
daysOfSalesOutstanding,72.220658,57.821837,69.433084,53.052971,23.059166,24.040824,55.415825,2.652498e+02,47.844988,69.484904,80.081007
daysOfInventoryOutstanding,2407.488640,104.590748,32.537220,5.428205,56.672777,55.261314,174.646066,0.000000e+00,92.698693,9.702365,30.295490
operatingCycle,2477.654353,160.583311,99.774785,57.989708,79.731943,78.147022,230.061890,4.191043e+01,116.268742,75.008328,110.376497
daysOfPayablesOutstanding,1441.151306,60.486827,91.205036,57.016142,63.800709,71.875439,263.644552,5.877090e+02,96.118258,117.018419,77.829820
cashConversionCycle,1037.235821,100.096484,8.569749,0.973567,15.931235,6.271583,-33.582662,-5.457986e+02,64.129108,-42.010091,32.546677
grossProfitMargin,0.336306,0.559349,0.554458,0.576690,0.434156,0.288841,0.494100,2.496069e-01,0.263656,0.562971,0.202370
operatingProfitMargin,0.163697,0.215636,0.315216,0.257278,0.189128,0.100446,0.173183,2.016690e-01,0.134265,0.335770,0.288375


In [ ]:
pprint(sector_ratios)

{'Communication Services': {'assetTurnover': 0.5841229887022882,
                            'capitalExpenditureCoverageRatio': -3.795947556378646,
                            'cashConversionCycle': 0.973566868217532,
                            'cashFlowCoverageRatios': 2.1119444095944946,
                            'cashFlowToDebtRatio': 2.1258851570686104,
                            'cashPerShare': 100.01984386028384,
                            'cashRatio': 0.5251751191272669,
                            'companyEquityMultiplier': 1.919526139257946,
                            'currentRatio': 2.2987853613169404,
                            'daysOfInventoryOutstanding': 5.428204984074305,
                            'daysOfPayablesOutstanding': 57.01614161895842,
                            'daysOfSalesOutstanding': 53.05297105335102,
                            'debtEquityRatio': 0.9051454354385833,
                            'debtRatio': 0.4345950139731849,
                    

In [ ]:
pprint(subIndustry_ratios)

{'Advertising': {'assetTurnover': 0.5078525914233388,
                 'capitalExpenditureCoverageRatio': -6.3027862723061805,
                 'cashConversionCycle': -143.8896384113787,
                 'cashFlowCoverageRatios': 0.3540013349700476,
                 'cashFlowToDebtRatio': 0.3540013349700476,
                 'cashPerShare': 17.591737132857475,
                 'cashRatio': 0.315629905482558,
                 'companyEquityMultiplier': 6.7055173990540595,
                 'currentRatio': 1.0027948038584036,
                 'daysOfInventoryOutstanding': 0.0,
                 'daysOfPayablesOutstanding': 383.01243835000435,
                 'daysOfSalesOutstanding': 239.1227999386257,
                 'debtEquityRatio': 5.695717000898579,
                 'debtRatio': 0.8460614871685042,
                 'dividendPaidAndCapexCoverageRatio': -11.043117766190218,
                 'dividendPayoutRatio': 0.4174148529149119,
                 'dividendYield': 0.034357204869747

In [39]:
def get_annual_ticker_ratios(year):
    f = 'data/financial_ratios'
    ratios = pd.read_csv(os.path.join(f, f'Annual-{year} Ticker Ratios.csv'))
    
    return ratios

ticker_ratios = get_annual_ticker_ratios('2020')

In [42]:
ticker_df = get_ticker_data('AAPL')
y = ticker_df.loc[:'2021', 'adjclose'].resample('Y') \
                                        .last() \
                                        .pct_change() \
                                        .dropna()
y

1981-12-31   -0.351652
1982-12-31    0.350292
1983-12-31   -0.184103
1984-12-31    0.194868
1985-12-31   -0.244636
1986-12-31    0.840919
1987-12-31    1.084447
1988-12-31   -0.033519
1989-12-31   -0.115624
1990-12-31    0.235010
1991-12-31    0.323313
1992-12-31    0.069115
1993-12-31   -0.504428
1994-12-31    0.352113
1995-12-31   -0.173362
1996-12-31   -0.345097
1997-12-31   -0.371255
1998-12-31    2.119031
1999-12-31    1.511454
2000-12-31   -0.710638
2001-12-31    0.472267
2002-12-31   -0.345661
2003-12-31    0.491276
2004-12-31    2.013571
2005-12-31    1.232609
2006-12-31    0.180137
2007-12-31    1.334748
2008-12-31   -0.569114
2009-12-31    1.469010
2010-12-31    0.530679
2011-12-31    0.255580
2012-12-31    0.325669
2013-12-31    0.080695
2014-12-31    0.406225
2015-12-31   -0.030137
2016-12-31    0.124804
2017-12-31    0.484643
2018-12-31   -0.053902
2019-12-31    0.889578
2020-12-31    0.823067
2021-12-31    0.346482
Freq: A-DEC, Name: adjclose, dtype: float64

In [ ]:
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
hist = table[1]
# hist['Date'] = pd.to_datetime(hist['Date'])
hist.head()

Date  Added                        Removed  \
            Date Ticker               Security  Ticker   
0  June 21, 2022    KDP       Keurig Dr Pepper  UA/UAA   
1  June 21, 2022     ON       ON Semiconductor    IPGP   
2   June 8, 2022   VICI        Vici Properties    CERN   
3  April 4, 2022    CPT  Camden Property Trust    PBCT   
4  March 2, 2022    MOH      Molina Healthcare    INFO   

                              \
                    Security   
0               Under Armour   
1              IPG Photonics   
2                     Cerner   
3  People's United Financial   
4                 IHS Markit   

                                              Reason  
                                              Reason  
0                   Market capitalization change.[5]  
1                   Market capitalization change.[5]  
2  S&P 500 constituent Oracle Corp. acquired Cern...  
3  S&P 500 constituent M&T Bank Corp. acquired Pe...  
4  S&P 500 constituent S&P Global Inc. acquired I...

In [ ]:
added = hist['Date'].join(hist['Added'])
added['inSPY'] = True
added['Date'] = pd.to_datetime(added['Date'])
added.head()

,Date,Ticker,Security,inSPY
0,2022-06-21,KDP,Keurig Dr Pepper,True
1,2022-06-21,ON,ON Semiconductor,True
2,2022-06-08,VICI,Vici Properties,True
3,2022-04-04,CPT,Camden Property Trust,True
4,2022-03-02,MOH,Molina Healthcare,True


In [ ]:
removed = hist['Date'].join(hist['Removed'])
removed['inSPY'] = False
removed['Date'] = pd.to_datetime(removed['Date'])
removed.head()

,Date,Ticker,Security,inSPY
0,2022-06-21,UA/UAA,Under Armour,False
1,2022-06-21,IPGP,IPG Photonics,False
2,2022-06-08,CERN,Cerner,False
3,2022-04-04,PBCT,People's United Financial,False
4,2022-03-02,INFO,IHS Markit,False


In [ ]:
hist = pd.concat([added, removed]).sort_values('Date', ascending=False)
hist.set_index('Date', inplace=True)
hist.dropna(inplace=True)
hist.head()

,Ticker,Security,inSPY
Date,,,
2022-06-21,KDP,Keurig Dr Pepper,True
2022-06-21,ON,ON Semiconductor,True
2022-06-21,IPGP,IPG Photonics,False
2022-06-21,UA/UAA,Under Armour,False
2022-06-08,CERN,Cerner,False


In [ ]:
msft = yf.Ticker("MSFT")

# get stock info
msft.info

{'zip': '98052-6399',
 'sector': 'Technology',
 'fullTimeEmployees': 221000,
 'longBusinessSummary': 'Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide. The company operates in three segments: Productivity and Business Processes, Intelligent Cloud, and More Personal Computing. The Productivity and Business Processes segment offers Office, Exchange, SharePoint, Microsoft Teams, Office 365 Security and Compliance, Microsoft Viva, and Skype for Business; Skype, Outlook.com, OneDrive, and LinkedIn; and Dynamics 365, a set of cloud-based and on-premises business solutions for organizations and enterprise divisions. The Intelligent Cloud segment licenses SQL, Windows Servers, Visual Studio, System Center, and related Client Access Licenses; GitHub that provides a collaboration platform and code hosting service for developers; Nuance provides healthcare and enterprise AI solutions; and Azure, a cloud platform. It also offers enterprise

In [ ]:
df = pd.read_csv('data/SPY-Historical.csv')
tickers = df['Ticker'].to_list()
len(tickers)

594

In [3]:
securities_list = fa.available_companies('a812dc7d35321ed0502ea50d8dc71be9')

In [5]:
securities_list['type'].unique()

array(['etf', 'stock', 'trust', 'fund'], dtype=object)

In [6]:
securities_list[securities_list['type'] == 'etf']

,name,price,exchange,exchangeShortName,type
symbol,,,,,
SPY,SPDR S&P 500 ETF Trust,396.91,New York Stock Exchange Arca,AMEX,etf
GDX,VanEck Gold Miners ETF,24.19,New York Stock Exchange Arca,AMEX,etf
EEM,iShares MSCI Emerging Markets ETF,38.25,New York Stock Exchange Arca,AMEX,etf
XLF,Financial Select Sector SPDR Fund,33.74,New York Stock Exchange Arca,AMEX,etf
EFA,iShares MSCI EAFE ETF,60.49,New York Stock Exchange Arca,AMEX,etf
...,...,...,...,...,...
ASOL.SW,21Shares Solana staking ETP,19.08,Swiss Exchange,SIX,etf
AVAX.SW,21Shares Avalanche ETP,3.63,Swiss Exchange,SIX,etf
ABNB.SW,21Shares Binance BNB ETP,20.12,Swiss Exchange,SIX,etf
